Stock Calculator Automation
----------------
This automation allows you to create a simple webpage with an input form that allows to select a date and when submitted, you see a higest rising stock from a selected set of stocks. 


Needed imports

In [ ]:
from bspump.jupyter import *
from bspump.http.web.server import *
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import io
from datetime import date
import base64

Defining an autopipline, the core of the automation. We define its source, sink.

In [ ]:

today = datetime.now()

auto_pipeline(
    source=lambda app, pipeline: WebFormSource(app, pipeline, route="/",
    form_intro="""<h1> Select date range and click submit to see a visualization""",
    fields=[
        IntField("Day",default=today.day),
        IntField("Month",default=today.month),
        IntField("Year",default=today.year),
    ]),
    sink=lambda app, pipeline: WebSink(app, pipeline)
)

Pipeline section
----------------

Everything after this is rerun every time an event comes in. At run time, the `event` variable is automatically set with the value of the event that comes from the source.

We can do whatever transformations we please, and then, by setting `event` at the end of the notebook, the value of `event` will automatically be sent to the sink.

Here is a helper function of the output layout.

In [ ]:
def output_layout(parsed_date,top_stock_name,top_stock_change,img_data) -> str:
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Stock Performance</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                text-align: center;
                background-color: #f5f5f5;
                padding: 40px;
            }}
            h1 {{
                color: #333;
                margin-bottom: 20px;
            }}
            img {{
                border: 1px solid #ccc;
                max-width: 90%;
                height: auto;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
            }}
        </style>
    </head>
    <body>
        <h1>📈 Top Stock on {parsed_date}: {top_stock_name} (change {top_stock_change:.2f}%)</h1>
        <img src="data:image/jpeg;base64,{base64.b64encode(img_data.getvalue()).decode()}" alt="Stock Performance">
    </body>
    </html>
    """
    return html

In this last part we work with the actual event, It initially has the data from the web form and then we use it to calculate the data nad set the result as an HTML containing stylyzed by the output_layout function containing the results from the calculation and image of the result plot.

In [ ]:
# Global variables
STOCKS = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "META", "JPM", "NFLX", "AMD"]

HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        }

# --- Input processing ---
form = event["form"]
parsed_date_str = date(form["Year"], form["Month"], form["Day"]).strftime("%Y-%m-%d")
parsed_date = datetime.strptime(parsed_date_str, "%Y-%m-%d")
day_before = parsed_date - timedelta(days=1)
day_after = parsed_date + timedelta(days=1)

# --- Initialize change tracking ---
changes = {}

# --- Iterate over each ticker and fetch data ---
for ticker in STOCKS:
    # Convert datetime to UNIX timestamps
    start_ts = int(time.mktime(day_before.timetuple()))
    end_ts = int(time.mktime(day_after.timetuple()))

    # Prepare API request
    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}"
    params = {
        "period1": start_ts,
        "period2": end_ts,
        "interval": "1d"
    }

    try:
        response = requests.get(url, params=params, headers=HEADERS)
        data = response.json()

        result = data["chart"]["result"][0]
        timestamps = result["timestamp"]
        closes = result["indicators"]["quote"][0]["close"]

        # Convert timestamps to readable dates
        dates = [datetime.fromtimestamp(ts).strftime('%Y-%m-%d') for ts in timestamps]
        series = pd.Series(closes, index=dates)

        # Compute percent change if both dates exist in the series
        if parsed_date_str in series and day_before.strftime("%Y-%m-%d") in series:
            previous_close = series[day_before.strftime("%Y-%m-%d")]
            current_close = series[parsed_date_str]
            if previous_close and current_close:
                percent_change = ((current_close - previous_close) / previous_close) * 100
                changes[ticker] = percent_change

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# --- Prepare output if data is available ---
if changes:
    df = pd.DataFrame(list(changes.items()), columns=["Ticker", "Change (%)"])
    df.sort_values("Change (%)", ascending=False, inplace=True)

    # Plotting
    ax = df.plot.bar(
        x="Ticker",
        y="Change (%)",
        title=f"Stock Change (%) on {parsed_date_str}",
        xlabel="Ticker",
        ylabel="Change (%)",
        grid=True,
        figsize=(10, 7)
    )

    # Save plot to buffer
    buf = io.BytesIO()
    ax.figure.savefig(buf, format="jpeg")
    img_data = buf

    # Get top stock info
    top_stock = max(changes, key=changes.get)
    top_stock_name = top_stock
    top_stock_change = changes[top_stock]

    # Build and set response
    response = output_layout(
        parsed_date=parsed_date_str,
        top_stock_name=top_stock_name,
        top_stock_change=top_stock_change,
        img_data=img_data
    )

    event["response"] = response
    event["content_type"] = "text/html"

else:
    print("No data found. Market may have been closed.")